In [ ]:
# import
import numpy as np
import pandas as pd
from pathlib import Path
import chardet


In [ ]:
# Create path to CSV
gaming = Path("GamingStudy\Scrub_gaming_data.csv")

In [ ]:
# read in binary mode and detect encoding
# with open("../Scrub gaming data.csv", 'rb') as f:
#     result = chardet.detect(f.read())
    
# encoding = result['encoding']



In [ ]:
# Read data file with Pandas
# gaming_df = pd.read_csv(gaming, encoding = "ISO-8859-1")
gaming_df = pd.read_csv(gaming)
gaming_df["whyplay"].value_counts()
gaming_df

In [ ]:
gaming_df.columns

In [ ]:
clean_game = gaming_df[["S. No.","Game","Platform","Hours","earnings","whyplay","Gender","Age","Work","Degree","Residence","Playstyle","GAD_T","SWL_T","SPIN_T"]]
clean_game

In [ ]:
clean_game.to_csv("clean_game_data.csv")


In [ ]:
# Aliyu's Code Starts Here

In [ ]:
#Aliyu's code stops here

#Amy's code starts here

In [ ]:
#Amy's code starts here
#Andrew's code starts here

In [ ]:
#Andrew's code stops here
#Anna's code starts here

Question: Are gamers that play alone more anxious than those who play socially?

(Analysis completed by Anna Bitzer)

In [ ]:
#question specific dependencies
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
#filter dataset further to only relevant columns
playstyle_df = gaming_df[["S. No.", "Gender", "Age", "Playstyle", "GAD_T"]]
playstyle_df = playstyle_df.rename(columns={"S. No.": "Survey Number"})
playstyle_df = playstyle_df.set_index("Survey Number")

playstyle_df.head()

In [ ]:
#check for list of play styles
playstyle_df["Playstyle"].value_counts()

#keep only the top 5 playstyles
playstyle_filtered = playstyle_df[(playstyle_df["Playstyle"] == "Multiplayer - online - with real life friends")
                                          |(playstyle_df["Playstyle"] == "Multiplayer - online - with strangers")
                                          |(playstyle_df["Playstyle"] == "Multiplayer - online - with online acquaintances or teammates")
                                          |(playstyle_df["Playstyle"] == "Singleplayer")
                                          |(playstyle_df["Playstyle"] == "Multiplayer - offline (people in the same room)")]

all_playstyle = (len(playstyle_df))      
filtered_playstyle = (len(playstyle_filtered))

dropped_playstyle = all_playstyle - filtered_playstyle
percent_dropped = round((dropped_playstyle/all_playstyle) * 100)

print(f'For the scope of this analysis, only the top 5 playstyles were included. A total of {dropped_playstyle} data points ({percent_dropped}% of the total) were dropped.')


In [ ]:
#create a piechart to give a visual of the playstyle distribution
#calculate number of each playstyle
playstyle_breakdown = playstyle_filtered["Playstyle"].value_counts()
#plot, format percentages, format visuals, add title
playstyle_breakdown.plot(kind = "pie", title = "Breakdown of Playstyles",
                     autopct = "%1.1f%%", explode = (0.1, 0, 0, 0, 0), shadow = True, 
                     startangle = 140, ylabel = "")

plt.show()


In [ ]:
#calculate summary stats of GAD score for overall population (min, max, mean, median, std dev)
#p + stat denotes overall population result
p_min = playstyle_filtered["GAD_T"].min()
p_max = playstyle_filtered["GAD_T"].max()
p_mean = playstyle_filtered["GAD_T"].mean()
p_median = playstyle_filtered["GAD_T"].median()
p_std = playstyle_filtered["GAD_T"].std()

#add summary stats to a dataframe
p_GAD_summary = pd.DataFrame([{"Min. GAD Result": p_min, "Max. GAD Result": p_max,
"Mean GAD Result": p_mean, "Median GAD Result": p_median, "GAD Std. Dev.": p_std}])

#format dataframe
p_GAD_summary["Mean GAD Result"] = round(p_GAD_summary["Mean GAD Result"], 2)
p_GAD_summary["GAD Std. Dev."] = round(p_GAD_summary["GAD Std. Dev."], 2)

p_GAD_summary


In [ ]:
#calculate min, max, mean, median, std. dev. of GAD score for each playstyle
gad_summary = playstyle_filtered.groupby(["Playstyle"])
#ps + stat denotes specific playstyle
ps_min = gad_summary["GAD_T"].min()
ps_max = gad_summary["GAD_T"].max()
ps_mean = gad_summary["GAD_T"].mean()
ps_median = gad_summary["GAD_T"].median()
ps_std = gad_summary["GAD_T"].std()

#add summary stats to a dataframe
ps_GAD_summary = pd.DataFrame({"Min. GAD Result": ps_min, "Max. GAD Result": ps_max,
"Mean GAD Result": ps_mean, "Median GAD Result": ps_median, "GAD Std. Dev.": ps_std})

#format dataframe
ps_GAD_summary["Mean GAD Result"] = round(ps_GAD_summary["Mean GAD Result"], 2)
ps_GAD_summary["GAD Std. Dev."] = round(ps_GAD_summary["GAD Std. Dev."], 2)

ps_GAD_summary

In [ ]:
#generate boxplot with playstyle versus GAD score
fig1, ax1 = plt.subplots()
ps_list = ["Multiplayer - offline (people in the same room)", 
                "Multiplayer - online - with real life friends",
                "Multiplayer - online - with strangers", 
                "Multiplayer - online - with online acquaintances or teammates",
                "Singleplayer"]
#empty list to hold plot data in                
ps_plot = []

#select data for each playstyle
for playstyle in ps_list:
    to_plot = playstyle_filtered.loc[(playstyle_filtered["Playstyle"] == playstyle)]["GAD_T"]
    ps_plot.append(to_plot)

ax1.boxplot(ps_plot)
ax1.set_xticklabels(ps_list, rotation = 90)
ax1.set_ylabel("GAD Score")
plt.title("GAD Score Distribution by Playstyle")

plt.show()


In [ ]:
#notably, there are a lot of outliers for each Playstyle determine the list of outliers for each 
outliers = []

for playstyle in ps_list:
    #first, determine the quartiles
    quartiles = list(playstyle_filtered[(playstyle_filtered["Playstyle"] == playstyle)]["GAD_T"].quantile([0.25, 0.5, 0.75]))
    lower_q = quartiles[0]
    upper_q = quartiles[2]
    iqr = upper_q - lower_q
    #set boundaries to search for outliers
    upper_outliers = upper_q + (1.5*iqr)
    lower_outliers = lower_q - (1.5*iqr)
    #set variable for list of GAD score for specific playstyle
    gad_scores = playstyle_filtered[(playstyle_filtered["Playstyle"] == playstyle)]["GAD_T"]
    #determine how many outliers there are
    upper_outliers_count = gad_scores.loc[(gad_scores > upper_outliers)].count()
    lower_outliers_count = gad_scores.loc[(gad_scores < lower_outliers)].count()
    #determine total number of outliers
    outlier_total = upper_outliers_count + lower_outliers_count
    #determine what percentage of total results are considered outliers
    outlier_percent = round(outlier_total/(gad_scores.count()) *100)
    print(f'{playstyle} has {outlier_total} outliers ({outlier_percent}% of data points).')
    #reset outlier list before next playstyle
    outliers = []


In [ ]:
#the largest difference in mean GAD score is between the groups Multiplayer - offline (people in the same room) and Singleplayer
#use the Welch's t-test to determine if there is a significant difference between these two groups
multi_offline = playstyle_filtered[(playstyle_filtered["Playstyle"] == "Multiplayer - offline (people in the same room)")]["GAD_T"]
single = playstyle_filtered[(playstyle_filtered["Playstyle"] == "Singleplayer")]["GAD_T"]

stats.ttest_ind(multi_offline, single, equal_var = False)

Conclusion: The results of this data found that while all styles of playing video games were associated with similar average GAD scores, the total average GAD score of players who played alone was higher than players that played multiplayer. The mean GAD scores of players that played offline multiplayer, with real life friends (mean GAD = 4.21) and online multiplayer, with real life friends (mean GAD = 4.77) were below the overall population mean (GAD = 5.22), suggesting that less anxious players may be more comfortable playing with people they know in real life. The remaining playstyles multiplayer-online-with online acquaintances or teammates, multiplayer-online-with strangers and single player had mean GAD scores above the population mean, of 5.47, 5.57 and 5.80 respectively.

A Welch's t-test was performed to compare the group with the lowest mean GAD score of 4.21, gamers that played offline multiplayer, against the group with the highest mean GAD score of 5.80, gamers that played single player. It was found that there is a slightly significant difference between the two groups, with a p-value of 0.03. It should be noted that these two groups were by far the smallest of the playstyles analyzed - 5.8% of the population played single player, and only 0.4% of the population played offline multiplayer. 

The GAD data was not normally distributed, and was instead right skewed, with the median less than the mean for all playstyle categories. Results for all playstyles ranged from 0-21 (with the exception of the multiplayer-offline-friends in the same room group, which had a range of 0-20). There were a significant amount of high outliers for each group, bewteen 2-8% of data points were outside of the range of 1.5 times the IQR for each group. 

There were some limitations to this dataset. The majority of respondents to this dataset listed League of Legends as the primary video game that they played. As this is a multiplayer game, the distribution of the data is skewed away from single player gamers. A dataset with a more even breakdown of single player versus multiplayer gamers would allow a more accurate view of anxiety levels of those who game alone versus those who game with either online or real life friends.